# Declarative Workflows with Report/Form

Multi-step agent workflows using the `lionpride.work` module.

In [1]:
from dotenv import load_dotenv

load_dotenv()

from pydantic import BaseModel, Field

from lionpride import Session, iModel
from lionpride.work import Report, flow_report

## 1. Define Output Schemas

In [2]:
class Summary(BaseModel):
    text: str = Field(description="Brief summary")
    word_count: int = Field(description="Number of words")


class Analysis(BaseModel):
    key_points: list[str] = Field(description="Main points")
    sentiment: str = Field(description="positive, negative, or neutral")


class Result(BaseModel):
    title: str = Field(description="Report title")
    conclusion: str = Field(description="Final conclusion")

## 2. Define Report with Workflow

In [3]:
class TopicReport(Report):
    # Output schemas (filled during execution)
    summary: Summary | None = None
    analysis: Analysis | None = None
    result: Result | None = None

    # Workflow
    assignment: str = "topic -> result"
    form_assignments: list[str] = [
        "topic -> summary",
        "topic -> analysis",
        "summary, analysis -> result",
    ]

## 3. Explore the Report

In [4]:
report = TopicReport()
print(f"Input: {report.input_fields}")
print(f"Output: {report.output_fields}")
print(f"Forms: {len(list(report.forms))}")

Input: ['topic']
Output: ['result']
Forms: 3


In [5]:
# Type introspection
print(f"summary type: {report.get_field_type('summary')}")
print(f"analysis type: {report.get_field_type('analysis')}")
print(f"result type: {report.get_field_type('result')}")

summary type: <class '__main__.Summary'>
analysis type: <class '__main__.Analysis'>
result type: <class '__main__.Result'>


In [6]:
# Initialize and check workable forms
report.initialize(topic="The impact of remote work on productivity")
print(f"Available: {list(report.available_data.keys())}")
print(f"Workable: {[f.assignment for f in report.next_forms()]}")

Available: ['topic']
Workable: ['topic -> summary', 'topic -> analysis']


## 4. Run the Workflow

In [7]:
# Setup
model = iModel(provider="openai", model="gpt-4.1-mini", name="gpt4mini")
session = Session(default_generate_model=model)
branch = session.create_branch(
    name="workflow",
    capabilities={"summary", "analysis", "result"},
    resources={"gpt4mini"},
)

In [8]:
# Fresh report
report = TopicReport()
report.initialize(topic="The impact of remote work on productivity")

# Execute
result = await flow_report(session, branch, report, verbose=True)

Compiled 3 forms to graph
Graph: 3 nodes, 3 edges
Pre-allocated branches for 3 operations
Operation 9cd73d32 prepared with 1 context items
Executing operation: 9cd73d32
Operation b29b62d7 waiting for 1 graph dependencies
Operation c9d9ff15 waiting for 2 graph dependencies


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Operation 9cd73d32 status after invoke: completed
  Execution error: None
Completed operation: 9cd73d32
Operation b29b62d7 prepared with 2 context items
Executing operation: b29b62d7
Operation 'summary' completed


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Operation b29b62d7 status after invoke: completed
  Execution error: None
Completed operation: b29b62d7
Operation c9d9ff15 prepared with 3 context items
Executing operation: c9d9ff15
Operation 'analysis' completed


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Operation c9d9ff15 status after invoke: completed
  Execution error: None
Completed operation: c9d9ff15
Operation 'result' completed


In [9]:
# Results
print(f"Progress: {report.progress}")
print(f"Complete: {report.is_complete()}")
print(f"\nDeliverable: {list(result.keys())}")

Progress: (3, 3)
Complete: True

Deliverable: ['result']


In [10]:
# Inspect output
if result.get("result"):
    r = result["result"]
    print(f"Title: {r.title}")
    print(f"Conclusion: {r.conclusion}")

Title: Summary and Analysis Report
Conclusion: The summary and analysis provide a concise overview and a neutral evaluation of the topic, effectively highlighting the key points without bias.
